In [33]:
import importlib
import utils.data_loader
import utils.tokenize
import models.embedding

#For reloading
importlib.reload(utils.data_loader)
importlib.reload(utils.tokenize)
importlib.reload(models.embedding)

<module 'models.embedding' from '/Users/bimalmgr/Downloads/deep_learning/transformers/trnsfmr_from_scratch/models/embedding.py'>

In [34]:
from utils.data_loader import DataLoader
from utils.tokenize import Tokenizer
from data import en,es
from models.embedding import Embeddings
from data import en,es
import torch
import torch.nn as nn

In [38]:
def reload_modules():
    %load_ext autoreload
    %autoreload 2

In [41]:
def main():
    reload_modules()
    print("This is the implementation of transformer from scratch in PyTorch.........!!")
    tokenizer = Tokenizer()
    data_loader = DataLoader(
        tokenize_source=tokenizer.tokenize_source,
        tokenize_target=tokenizer.tokenize_target
    )
    train_set,valid_set,test_set = data_loader.make_dataset(es,en)
    vocab_src,vocab_trg = data_loader.build_vocab(train_set,valid_set,test_set)
    # for word,index in vocab_trg.get_stoi().items():
    #     print(word,index)
   
    print("Source vobularies:",vocab_src)
    embed = nn.Embedding(len(vocab_src),3)
    print(embed(torch.tensor(vocab_src.get_stoi()['víctimas'])))
main()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
This is the implementation of transformer from scratch in PyTorch.........!!
Initializing the data loader........
Building Spanish Vocabulary.....
Done Building Spanish Vocabulary!
Buidling English Vocabulary.........
Done building English Vocabulary!
Source vobularies: Vocab()
tensor([-0.6753,  0.3612,  0.2072], grad_fn=<EmbeddingBackward0>)
